## VST Demo

This notebook showcases `vst` version `0.1.0` with updated functions and syntax to demonstrate a basic but complete model analysis workflow, using a simple toy model:
1. Partial model calibrations using variant model files (generation of these is manual for now but could easily be automated)
2. Full model calibration, building on the results of previous partial calibrations
3. Calibration with MCMC to derive a sample of posterior parameter estimates
4. Semi-automated generation of additional files to run projections
5. Basic projections + parametric sensitivity (for uncertainty intervals) based on calibrated parameter estimates
6. Projections for policy analysis + sensitivity analysis

At the moment this notebook relies on simple import of `vst.py` (and by extension `vst_text.py`); proper module packaging will occur eventually. For documentation on the `vst` code, objects, and functions, see the `.py` file or `.ipynb` notebooks.

**NOTE:** this version of `vst` is *not* backward-compatible with existing scripts: 1) `Script` objects and use syntax have been substantially modified, and 2) newest Vengine no longer has the expiration popup to bypass, so `press` lines are no longer needed and have been removed.

In [ ]:
from vst import *

In [ ]:
# Set up control file (this could be done through an external file read in as JSON)

# Global control inputs
timelimit = 60
vgpath = "C:/Program Files/Vengine/Vengine.exe"  # Change this to your Vengine file path!
vspath = "C:/Program Files/Vensim/vendss64.exe"  # Change this to your Vensim file path!
samplefrac = 0.05

# Controlfile inputs
cf = {
    'basename': 'Test', 
    'simcontrol': {
        "model": "prey-predator.mdl", 
        "data": [
            "hudson-bay-lynx-hare.vdf"
            ], 
        "payoff": "prey-predator.vpd", 
        "sensitivity": "", 
        "optparm": "prey-predator.voc", 
        "changes": [], 
        "savelist": "", 
        "senssavelist": "PrPrSave.lst"
    }, 
    'runcmd': '', 
    'savecmd': ''    
}

In [ ]:
# Create logfile
log = f"{os.getcwd()}/log.txt"

# Start with partial model calibration of just prey-side parameters
prey = Script(cf, "hare", log, sfxs={'optparm': '_h', 'payoff': '_h'})
prey.compile_script(vgpath, log, subdir='Hare')  # Run this in subdirectory 'Hare'

# Continue with partial model calibration of just predator-side parameters
pred = Script(cf, "lynx", log, sfxs={'optparm': '_l', 'payoff': '_l'}, chglist=[('hare', '')])
pred.compile_script(vgpath, log, subdir='Lynx')  # Run this in subdirectory 'Lynx'

# Then do a calibration of the full model with all params
# Note use of chglist to read in previous .out files
both = Script(cf, "fullopt", log, chglist=[(['hare', 'lynx'], '')])
both.compile_script(vgpath, log, subdir='Full')

# Then run MCMC, reading in full optimisation results
mcmc = Script(cf, "fullMC", log, sfxs={'optparm': '_m'}, chglist=[('full', 'opt')])
mcmc.compile_script(vgpath, log, subdir='Full')

mcmc.downsample(samplefrac)  # Downsample MCMC result to get input for sensitivity

In [ ]:
# Set up alternate control file for sensitivity runs
senscf = cf.copy()
senscf['simcontrol']['sensitivity'] = f"{mcmc.runname}.vsc"  # Sensitivity control file
senscf['savecmd'] = 'SENS2FILE|!|!|T'  # Save full sensitivity output in Tidy format

# Create CIN file for projections, extending final time by 10
finaltime = get_value(cf['simcontrol']['model'], 'FINAL TIME')  # Dynamically read in final time
with open('Projection.cin', 'w') as f:
    f.write(f"FINAL TIME = {finaltime + 10}")  # Extend by 10 units for projections


In [ ]:
# Do a simple projection using estimated parameters
proj = Script(cf, "projbase", log, chglist=[('full', 'opt'), 'Projection.cin'], simtype='r')
# Note use of Vensim rather than Vengine for simple run; in practice could use either
proj.compile_script(vspath, log, subdir='Projections', vengine=False, check_funcs=[check_output])

# Then a sensitivity analysis using previous MCMC subsample
sens = Script(senscf, "sensbase", log, chglist=[('full', 'opt'), 'Projection.cin'], simtype='s')
sens.compile_script(vgpath, log, subdir='Projections', timelimit=300, check_funcs=[check_output])

# Then read in a 'Hunting' policy scenario and project effects
hunt = Script(cf, "projhunt", log, chglist=[('full', 'opt'), 'Projection.cin', 'Hunting.cin'], simtype='r')
hunt.compile_script(vspath, log, subdir='Projections', vengine=False, check_funcs=[check_output])

# And conduct sensitivity analysis on 'Hunting' scenario with MCMC subsample
shunt = Script(senscf, "senshunt", log, chglist=[('full', 'opt'), 'Projection.cin', 'Hunting.cin'], simtype='s')
shunt.compile_script(vgpath, log, subdir='Projections', timelimit=300, check_funcs=[check_output])


In [ ]:
000000000000000000000000000000000000000000000000000000000000000000000000
000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000